Timeseries Plot For Multiple Model Builds

In [ ]:
library(readr)
library(dplyr)
library(tidyr)
library(stringr)
library(purrr)
library(ggplot2)
library(ggthemr)
library(here)
library(lubridate)
ggthemr("fresh")
devtools::load_all()


In [ ]:
# The following types are being modelled:
# Erle - Alder - Aulne - Alnus
# Birke - Birch - Bouleau - Betula
# Gräser - Grasses - Graminées - Poaceae
# Ambrosia - Ragweed - Ambroisie - Ambrosia

species_all <- tibble(
  taxon = c(
    "Castanea",
    "Alnus",
    "Ulmus",
    "Cupressus",
    "Fraxinus",
    "Fagus",
    "Juglans",
    "Plantago",
    "Corylus",
    "Pinus",
    "Quercus",
    "Rumex",
    "Platanus",
    "Populus",
    "Poaceae",
    "Salix",
    "Betula",
    "Carpinus",
    "Urtica",
    "Taxus",
    "Picea",
    "Ambrosia"
  ),
  hirst_taxon = c(
    "kacasth0",
    "kaalnuh0",
    "kaulmuh0",
    "kacuprh0",
    "kafraxh0",
    "kafaguh0",
    "kajuglh0",
    "khplanh0",
    "kacoryh0",
    "kapinuh0",
    "kaquerh0",
    "khrumeh0",
    "kaplath0",
    "kapopuh0",
    "khpoach0",
    "kasalih0",
    "kabetuh0",
    "kacarph0",
    "khurtih0",
    "kataxuh0",
    "kapiceh0",
    "khambrh0"
  ),
  cosmo_taxon = c(
    NA_character_,
    "ALNU",
    NA_character_,
    NA_character_,
    NA_character_,
    NA_character_,
    NA_character_,
    NA_character_,
    NA_character_,
    NA_character_,
    NA_character_,
    NA_character_,
    NA_character_,
    NA_character_,
    "POAC",
    NA_character_,
    "BETU",
    NA_character_,
    NA_character_,
    NA_character_,
    NA_character_,
    "AMBR"
  ),
  fieldextra_taxon = c(
    NA_character_,
    "ALNU1",
    NA_character_,
    NA_character_,
    NA_character_,
    NA_character_,
    NA_character_,
    NA_character_,
    NA_character_,
    NA_character_,
    NA_character_,
    NA_character_,
    NA_character_,
    NA_character_,
    "POAC1",
    NA_character_,
    "BETU1",
    NA_character_,
    NA_character_,
    NA_character_,
    NA_character_,
    "AMBR1"
  )
)

species <- species_all %>%
  filter(taxon %in% c("Alnus", "Ambrosia", "Betula", "Poaceae"))

stations <-
  tibble(
    hirst_station = c(
      "PDS",
      "PBU",
      "PMU",
      "PBS",
      "PZH",
      "PLZ",
      "PBE",
      # "PPY",
      "PNE",
      "PVI",
      "PLS",
      "PGE",
      "PCF",
      "PLO",
      # "BLR",
      "PLU"
    ),
    station = c(
      "Wolfgang",
      "Buchs",
      "Münsterlingen",
      "Basel",
      "Zürich",
      "Luzern",
      "Bern",
      # "Payerne",
      "Neuchâtel",
      "Visp",
      "Lausanne",
      "Genève",
      "La-Chaux-de-Fonds",
      "Locarno",
      # "Balerna",
      "Lugano"
    ),
    cosmo_station = c(
      "CHDAVO",
      "CHBUCH",
      "CHMUEN",
      "CHBASE",
      "CHZUER",
      "CHLUZE",
      "CHBERN",
      # NA_character_,
      "CHNEUC",
      "CHVISP",
      "CHLAUS",
      "CHGENE",
      "CHLACH",
      "CHLOCA",
      # NA_character_,
      "CHLUGA"
    )
  ) %>%
  arrange(hirst_station)

In [ ]:
data_dwh <- import_data_dwh(paste0(here(), "/data/dwh/pollen_dwh_hourly.txt"))
data_original <- import_data_cosmo("/scratch/sadamov/wd/20_all_original/cosmo_output_poac_2020.txt", type = "Operational - C1")
data_osm <- import_data_cosmo("/scratch/sadamov/wd/20_alnu_osm/mod_pollen_combined.txt", type = "Operational - C1E")
data_realtime <- import_data_cosmo("/scratch/sadamov/wd/20_alnu_assim_v1/mod_pollen_combined.txt", type = "With Real-Time Data")
data_sdes <- import_data_cosmo("/scratch/sadamov/wd/20_alnu_fit_sdes_alnu_median/mod_pollen_combined.txt", type = "With Phenology-Update")
data_realtime_sdes <- import_data_cosmo("/scratch/sadamov/wd/20_alnu_fit_sdes_all/mod_pollen_combined.txt", type = "With Real-Time Data and Phenology-Update")



In [ ]:
data <- bind_rows(
  # data_original,
  data_dwh,
  data_osm,
  # data_realtime,
  data_sdes %>% mutate(value = value * 0.6),
  data_realtime_sdes %>% mutate(value = value * 0.6)
)


In [ ]:
map(stations$station, ~ 
  data %>%
    filter(taxon == "Alnus",
          station == .x,
          between(datetime, as_datetime("2020-01-01 00:00:00"), as_datetime("2020-04-01 00:00:00"))) %>%
          group_by(date, taxon, station, type) %>%
          summarise(value = mean(value, na.rm = TRUE)) %>%
    ggplot(aes(x = date, y = value, col = type)) +
    geom_line(alpha = 0.6) +
    geom_point(aes(shape = type), alpha = 0.4) +
    scale_color_manual(values = c("#404040", "#76d3de", "#f4b258", "#ec7d64", "#52cdb5", "#bb809a")) +
    # scale_color_manual(values = swatch()[c(3, 2, 4, 5, 9, 7)]) +
    theme(legend.position="bottom", legend.title=element_blank()) +
    xlab("2020") +
    ylab("Daily Alder Concentration [m^-3]") +
    ggtitle(paste0("Daily Grasses Concentrations in ", .x, " - 2020 Season")))

# ggsave(paste0(here(), "/vignettes/figures/timeseries.png"), gg_timeseries, width = 24, height = 13.5, dpi = 300, units = c("cm"))